# Hubbard Model Part I - Dhilan Nag

In [2]:
from itertools import permutations as perm

#Model parameters
siteNum = int(input("# of sites: "))
upNum = int(input("# of spin up electrons: "))
downNum = int(input("# of spin down electrons: "))

# of sites: 2
# of spin up electrons: 1
# of spin down electrons: 1


In [3]:
#Create bit strings

#Spin up
upStrings = []
for i in range(upNum):
    upStrings.append("1")
for i in range(siteNum-upNum):
    upStrings.append("0")

upStrings = list(set(perm(upStrings,siteNum)))

#Spin down
downStrings = []
for i in range(downNum):
    downStrings.append("1")
for i in range(siteNum-downNum):
    downStrings.append("0")

downStrings = list(set(perm(downStrings,siteNum)))

#Set up string output for bit strings    
output = "|"
for up in upStrings:
    for down in downStrings:
        for b in up:
            output += b+","
        output = output[0:len(output)-1]+">|"
        for b in down:
            output += b+","
        output = output[0:len(output)-1]+">\n|"
output = output[0:len(output)-1]
    

print(output)

|0,1>|0,1>
|0,1>|1,0>
|1,0>|0,1>
|1,0>|1,0>



# Hubbard Model Part II

In [8]:
from numpy import zeros as emptyM #empty matrix

sampleHilb = [[1,0,1,0],
              [0,1,0,1],
              [0,1,1,0],
              [1,0,0,1],] # 2 sites, half filled

# sampleHilb = [[1,1,1,0],
#               [1,1,0,1]]

#Declare parameters
t = 1
U = 1
hilbertSpace = sampleHilb #change from sampleHilb

# Create empty Hamiltonian matrix
ham = emptyM([len(hilbertSpace),len(hilbertSpace)])

# Onsite term------

#Counting Operator for individual bitstring
def countingOp(bitString, spin, site):
    # reduce bitstring to sites in question
    if(spin == "u"):
        siteFind = bitString[0:siteNum]
    else:
        siteFind = bitString[siteNum:]
    # analyze fermion presence at site
    if(siteFind[site] == 0):
        return(0)
    else:
        return(1)

#Compute coefficient for individual onsite term ket
def computeOnsiteTerm(bitString, numSites, U):
    coefficient = 0
    for n in range(numSites):
        if(countingOp(bitString,"u",n) == 1 and countingOp(bitString,"d",n) == 1):
            coefficient += 1
    return [coefficient*U,bitString]

#onsite terms for each bit string
#format: list index --> tuple [coefficient, bitString]
        #list index represents bitString index in hilbert space
u_terms = []
for bitString in hilbertSpace:
    u_terms.append(computeOnsiteTerm(bitString, siteNum, U))

# Hopping term------

#format: list index --> tuple [ [coefficient, modified up bitString] , [coefficient, modified down bitString] ]
t_terms = []
for ind, bitString in enumerate(hilbertSpace):
    for i in range(siteNum-1):
        coefficient = 0
        newB = bitString
        j = i+1
        #address edge cases
    #     if(j == siteNum):
    #         j = 0

        t_terms.append([[0],[0]])
        #Spin up j annihilate, spin up i create
        if(countingOp(newB,"u",j) == 1 and countingOp(newB,"u",i) == 0):
            newB[j] = 0
            newB[i] = 1
            coefficient = 1
            t_terms[ind][0] = [coefficient*t, newB]

        #Spin down j annihilate, spin down i create
        if(countingOp(newB,"d",j) == 1 and countingOp(newB,"d",i) == 0):
            newB[j+siteNum] = 0
            newB[i+siteNum] = 1
            coefficient = 1
            t_terms[ind][1] = [coefficient*t, newB]



# inner product function
def innerProd(b,k):
    for i in range(len(k)):
        if(k[i] != b[i]):
            return(0)
    return(1)


c = 0
# compute all inner products (u term)
for i in range(len(hilbertSpace)):
    for j in range(len(hilbertSpace)):
        if(u_terms[j][0] != 0):
#             print(hilbertSpace[j])
#             print(u_terms[i][1])
#             print()
            ham[i,j] = u_terms[i][0]*innerProd(hilbertSpace[j], u_terms[i][1]) #<j|i>
        else:
            ham[i,j] = 0

print(u_terms)
print(t_terms)
print(ham)

[[1, [1, 0, 1, 0]], [1, [1, 0, 1, 0]], [0, [1, 0, 1, 0]], [0, [1, 0, 1, 0]]]
[[[0], [0]], [[1, [1, 0, 1, 0]], [1, [1, 0, 1, 0]]], [[1, [1, 0, 1, 0]], [0]], [[0], [1, [1, 0, 1, 0]]]]
[[1. 1. 0. 0.]
 [1. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
